# analysis of inertial partcles in daily averaged field of NWES
In this notebook we analyse the effect of MR advection on the trajectory of plastics.   
The particles were released in the daily flowfield in the NWES from the NWES dataset (https://doi.org/10.48670/moi-00054)    
They are released the 1st of every month from 1 september 2023 to 1 march 2024 and simulated for 30 days, where there   
location is saved very hour.  

We analyse 3 types of particles:
1. tracer particles
2. randomly displaced tracer particles
4. SMMR particles with flexible Rep

We compare 2-3 to 1 to study effect of MR vs tracer particles

We analyse:
- trajectory length 
- seperation distance
- denstity distribution 

And then compare to the results of the simulation with daily resolution

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

# import needed packages
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from decimal import Decimal

from datetime import datetime, timedelta
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/simulations")
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions import  make_PDF, Haversine
from analysis_functions_xr import trajectory_length, calc_tidal_av


from helper import create_filelist

# set plotscale
plt.style.use('../python_style_Meike.mplstyle')
colorlist=np.array(['black','c','orange','green','red','navy','pink'])
lineslist = ['-','-','--','--',':']


In [ ]:

#set directories and template filenames
base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'


basefile_MR_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}.zarr')

basefile_tracer = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')
basefile_tracer_random = (base_directory + 'tracer_random/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_d{d:04d}_{land_handling}.zarr')



particle_types = ['tracer','tracer_random','inertial_SM_drag_Rep']
basefiles = {'tracer':basefile_tracer,
             'tracer_random':basefile_tracer_random,
             'inertial_SM_drag_Rep':basefile_MR_Rep_drag

             }

legend_names = {'tracer':'tracer',
             'tracer_random':'displaced tracer',
             'inertial_SM_drag_Rep':'SMMR'
             }



In [ ]:
# set constants 
Rearth = 6371 * 10**3 # in k,
deg2rad = np.pi / 180.

#for simulation
runtime =  timedelta(days=30)# timedelta(days=10)
loc = 'NWES'
runtime=timedelta(30)
nparticles = 51548# 88347 # 52511
land_handling = 'anti_beaching'
coriolis = True
starttimes = np.array([datetime(2023, 9, 1, 0, 0, 0, 0)]) #,
                    #    datetime(2023, 10, 1, 0, 0, 0, 0),
                    #    datetime(2023, 11, 1, 0, 0, 0, 0) ,
                    #    datetime(2023, 12, 1, 0, 0, 0, 0) ])
                       #datetime(2024, 1, 1, 0, 0, 0, 0),
                       #datetime(2024, 2, 1, 0, 0, 0, 0)])


chunck_time = 100
years = np.array([2023,2024])
months = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
                      
#for particle
displacement = 300 #m


B = 0.68
tau = 2994.76 #2759.97

## Import data

In [ ]:
data = {}

traj = np.arange(0,nparticles,1)
#dictionary
data = {}
for pt in particle_types:
    data[pt]={}
    print(pt)
    for coriolis in [True]:
        data[pt][coriolis]={}
        for starttime in starttimes:
            print(starttime)
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            data[pt][coriolis][date]={}
            file = basefiles[pt].format(loc=loc,
                                                            y_s=starttime.year,
                                                            m_s=starttime.month,
                                                            d_s=starttime.day,
                                                            y_e=endtime.year,
                                                            m_e=endtime.month,
                                                            d_e=endtime.day,
                                                            B = int(B * 1000), 
                                                            tau = int(tau ),
                                                            land_handling = land_handling, 
                                                            cor_on = coriolis,
                                                            particle_type = pt,
                                                            d = displacement)
            print(file)
            ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)

            data[pt][coriolis][date]= ds 



In [ ]:
NWES_start2023_09_01_tres_daily_end2023_10_01_RK4_anti_beaching.zarr.

In [ ]:
# plot selection of trajectories

year=2023
month=9
date = f'{year:04d}/{month:02d}'

coriolis = True
fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
idstart = 1234 # 47584#98
idstep = 1
idend = idstart+1 * idstep

ax.add_feature(cart.feature.LAND, facecolor='lightgrey')
for pt, color, line in zip(particle_types,colorlist, lineslist):
        ax.plot(data[pt][coriolis][date].lon[idstart,0::1].T,
                data[pt][coriolis][date].lat[idstart,0::1].T,
                line,
                color=color);

legend = [legend_names[pt ]for pt in particle_types]
ax.legend(legend,loc=(1,0.2))

# for pt, color,line in zip(particle_types,colorlist,lineslist):
#         ax.plot(data[pt][coriolis][date].lon[idstart:idend:idstep,0::1].T,
#                 data[pt][coriolis][date].lat[idstart:idend:idstep,0::1].T,
#                 line,
#                 color=color);




ax.plot(data[pt][coriolis][date].lon[idstart:idend:idstep,0],
        data[pt][coriolis][date].lat[idstart:idend:idstep,0],
        '>',
        color='green',
        zorder=20)




## Along track displacement

In [ ]:
# along track distance
for pt in particle_types: # loop over all particle types
    for starttime in starttimes: # loop over all starttime
        year = starttime.year
        month = starttime.month
        date = f'{year:04d}/{month:02d}'
        traj_length = trajectory_length(data[pt][coriolis][date].lon, data[pt][coriolis][date].lat)
        data[pt][coriolis][date] = data[pt][coriolis][date].assign(traj_length = traj_length)

    


In [ ]:
# plot total trajectory lengh pdf
# color_particles = {'tracer':'navy','inertial':'firebrick','inertial_SM':'green'}
fig,ax=plt.subplots()
legend=[]
Tmax=720
for pt, color, line in zip(particle_types[0:1],colorlist, lineslist):
    
    # Extract DataArray objects from the dictionary
    data_arrays = list(data[pt][coriolis].values())

    # Combine DataArray objects along a new dimension
    combined_data = xr.concat(data_arrays, dim="starting_days")
    # Calculate the average along the new dimension
    average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

    bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),401, norm=True)
    ax.plot(bins,pdf,line,color=color)


legend = [legend_names[pt ]for pt in particle_types]
ax.legend(legend)
ax.set_ylabel("PDF")
ax.set_xlabel("trajectory length (km)")
# ax.set_xlim(0,2500)

In [ ]:
fig, ax = plt.subplots()
ref = 'tracer'

particle_types_compare = [pt for pt in particle_types if pt != ref]
Tmax=400

data_arrays_ref = list(data[ref][coriolis].values())
combined_data_ref = xr.concat(data_arrays_ref, dim="starting_days")

for pt, color, line in zip(particle_types_compare, colorlist[1:],lineslist[1:]):
    # Extract DataArray objects from the dictionary
    data_arrays = list(data[pt][coriolis].values())

    # Combine DataArray objects along a new dimension
    combined_data = xr.concat(data_arrays, dim="starting_days") 
    diff = combined_data.traj_length - combined_data_ref.traj_length
    # Calculate the average along the new dimension
    average_data = diff.mean(dim="starting_days",skipna=True)
    # test = test[~np.isnan(test)]
    # test = test[test > -1]
    bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten() ,10001, norm=True, min = -500, max =500)
    ax.plot(bins,pdf,line,color=color)
    # ax.set_xscale('log')
    # ax.set_yscale('log')
    ax.set_xlim(-5,5)